<h3>Build up the network</h3>
Your Genesis cloud API token needs to be set in the environment variable GC_API_TOKEN, the name of your SSH key should be set in the variable GC_SSH_KEY_NAME. If you are sure that you won't accidently push them to github, you can also overwrite them in here.

In [4]:
import os
from build_docker_image import *
from start_workers import *

In [5]:
ssh_key = os.getenv('GC_SSH_KEY_NAME')
api_token = os.getenv('GC_API_TOKEN')
no_workers = 1

gateway_instance = build_image(ssh_key, api_token)

Creating instance 21e4c7d4-dfac-4db4-b16d-f9391199be89
instance is starting ...
public ip is: 194.61.20.175
Used image snapshot nvidia+docker for installation.


In [6]:
nodes = start_workers(gateway_instance['ip'], api_token, ssh_key, no_workers)

Launching Alice as a worker node on a separate instance.
Creating instance b93fb3a3-3da1-4530-b247-43fa7fefbd9f
instance is starting ...
public ip is: 194.61.21.226
running startup script ...


TODO: The feedback system that figures out, when what module has finished installing is still faulty. In case of doubt check /var/log/cloud-init-output.log and rerun the docker-compose up.

<h3>Import all the required pysyft</h3>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import syft as sy
from syft.grid.public_grid import PublicGridNetwork
from syft.workers.node_client import NodeClient
hook = sy.TorchHook(torch)
sy.hook.local_worker.is_client_worker = False

In [2]:
GRID_PORT = '5000'
NODE_PORT = "3000"

In [4]:
gateway_instance = {'ip':'194.61.20.175'}
class node:
    def __init__(self,ip):
        self.ip = ip
nodes = [node('194.61.21.226')]

In [7]:
a = torch.tensor([1,2,3])
a_ptr = a.send(alice)
a_ptr = a.send(brutus)
a_ptr.get()

tensor([1, 2, 3])